In [2]:
import os

import anndata
import anndata as ad
from pathlib import Path
import os

# Loading data

In [ ]:
x_hat_s = anndata.read_h5ad(Path(os.getcwd()).parents[0] / "data" / "processed" / "mouse1_slice153_x_hat_s.h5ad")


# Method
## Calculate Ligand and Receptor score

## Calculate Ligand and Receptor Pair Score

## Calculate TF activity

## Calculate Intracellular score